## 5.2 Optimización de Modelos (Global Max Pooling)

En el notebook anterior decidimos optimizar los siguientes modelos:

 - Rank=1, Name=**SVC-linear-0.4**, Score=**0.9877151685937721 (+/- 0.003806562050586554)**
 - Rank=4, Name=**PassiveAggressiveClassifier**, Score=**0.9873060780235428 (+/- 0.005755384960731155)**
 - Rank=11, Name=**SGDClassifier**, Score=**0.986486033144723 (+/- 0.005575110872674419)**
 - Rank=14, Name=**KNeighborsClassifier-1**, Score=**0.9833439566419908 (+/- 0.007040876080775306)**
 
Para optimizar cada modelo, usaremos GridSearchCV, el cual es un método para automáticamente encontrar los mejores parámetros de un modelo, con base en una métrica objetivo que, en nuestro caso, es la exactitud (_accuracy_).

Las iniciales CV significan que usaremos _cross-validation_ o validación cruzada para estar verdaderamente seguros del desempeño del modelo. Como sudió en la fase de Evaluación de Modelos, haremos 10 validaciones cruzadas por modelo.

Como podemos esperar, este proceso tomará algo de tiempo, dado que no sólo entrenaremos muchos modelos, sino que cada variación necesitará de otros 10 modelos, debido al proceso de validación cruzada.

Empecemos definiendo las funciones auxiliares que usaremos para optimizar.
 

### Datos

Esta función nos retornará los datos relevantes necesatios para llevar a cabo la optimización de los modelos.

In [1]:
import numpy as np

def load_dataset():
    X = np.load('global_max_features.npy')
    y = np.load('labels.npy')
    
    return X, y

### Modelos

Esta función devuelve un dict de dicts que contiene el modelo a ser optimizado, junto con la grilla de parámetros para hacerlo.

In [2]:
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

def define_models(models=dict()):
    models['SGDClassifier'] = {
        'model': SGDClassifier(),
        'parameters': {
            'loss': ('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'),
            'penalty': ('none','l2', 'l1'),
            'alpha': (1e-3, 5e-3, 1e-4, 1e-2, 5e-3),
            'fit_intercept': (True, False)
        }
    }
    
    models['KNeighborsClassifier-1'] = {
        'model': KNeighborsClassifier(n_neighbors=1),
        'parameters': {
            'weights': ('uniform', 'distance'),
            'algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute')
        }
    }
    
    models['SVC-linear-0.4'] = {
        'model': SVC(kernel='linear', C=0.4),
        'parameters': {
            'probability': (True, False),
            'decision_function_shape': ('ovo', 'ovr'),
            'shrinking': (True, False)
        }
    }
    
    models['PassiveAggressiveClassifier'] = {
        'model': PassiveAggressiveClassifier(),
        'parameters': {
            'fit_intercept': (True, False),
            'C': (1, 0.1, 0.01, 1.5, 2),
            'loss': ('hinge', 'squared_hinge')
        }
    }
    
    print(f'Defined {len(models)} models.')

    return models

### Pipeline

Como en el notebook pasado, este pipeline es usado para pre-procesar los _features_ que le pasaremos al modelo.

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def make_pipeline(model):
    steps = [
        ('StandardScaler', StandardScaler()),
        ('MinMaxScaler', MinMaxScaler()),
        ('model', model)
    ]
    
    pipeline = Pipeline(steps=steps)
    
    return pipeline

### Búsqueda de Paráemtros

Aquí definimos un par de funciones auxiliares que llevarán a cabo la búsqueda de parámetros que optimizarán cada modelo. Al final del proceso, se imprimirán los mejores parámetros, junto con el puntaje/exactitud del mejor clasificador.

In [4]:
from sklearn.model_selection import GridSearchCV

def grid_search_model(X, y, model, folds, metric):
    m = model['model'] 
    parameters = model['parameters']
    
    classifier = GridSearchCV(m, parameters, cv=folds, scoring=metric, n_jobs=-1)
    pipeline = make_pipeline(classifier)
    pipeline.fit(X, y)
    
    return pipeline.steps[-1][1]

In [5]:
import warnings

def grid_search_models(X, y, models, folds=10, metric='accuracy'):
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        for model_name, model in models.items():
            m = grid_search_model(X, y, model, folds, metric)

            if m is not None:
                print(f'Best parameters for {model_name}: \n{m.best_params_}')
                print(f'Best model {metric}: {m.best_score_ * 100}%')
            else:
                print(f'{model_name}: error')
            
            print('----\n')

In [6]:
X, y = load_dataset()
models = define_models()
grid_search_models(X, y, models)

Defined 4 models.
Best parameters for SGDClassifier: 
{'alpha': 0.001, 'fit_intercept': False, 'loss': 'modified_huber', 'penalty': 'l2'}
Best model accuracy: 98.7849829351536%
----

Best parameters for KNeighborsClassifier-1: 
{'algorithm': 'auto', 'weights': 'uniform'}
Best model accuracy: 98.30716723549489%
----

Best parameters for SVC-linear-0.4: 
{'decision_function_shape': 'ovo', 'probability': True, 'shrinking': True}
Best model accuracy: 98.7849829351536%
----

Best parameters for PassiveAggressiveClassifier: 
{'C': 2, 'fit_intercept': False, 'loss': 'squared_hinge'}
Best model accuracy: 98.75767918088737%
----



## Conclusión

Estos son nuestros resultados:

 - **SVC-linear-0.4** pasa de 98.77151685937721% a 98.7849829351536%, con una mejora total de 0.013466075776378261%.
 - **PassiveAggressiveClassifier** pasa de 98.73060780235428% a 98.75767918088737%, con una mejora total de 0.027071378533079837%.
 - **SGDClassifier** pasa de 98.6486033144723% a 98.7849829351536%, con una mejora total de 0.1363796206812964%.
 - **KNeighborsClassifier-1** pasa de 98.33439566419908% a 98.30716723549489%, con una desmejora total de 0.027228428704191288%.
 
Observamos que a excepción de KNeighborsClassifier-1, todos los modelos muestran, al menos, una pequeña mejora en su exactitud, lo que significa que la búsqueda de parámetros fue exitosa.

Claramente, el modelo que más se benefició fue SGDClassifier, con una mejora total de  0.1363796206812964%. También alcanza el mejor puntaje, empatado con SVC-linear-0.4.

El mejor puntaje lo alcanzó SVC-linear-1.0: 99.35836177474403%.

Lamentablemente, KNeighborsClassifier-1 muestra un peor desempeño al final del procedimiento. Quizás jugar con otros parámetros o utilizar más vecinos de mejores resultados.

## Ideas

 - Probar otros modelos.
 - Probar más parámetros.
 - Probar reducción de dimensiones.
 - Usar más técnicas de ingeniería de _features_.
 - Usar _stacking_.
 
### Una nota final

No usamos la versión aplanada de los datos porque no cupo en la memoria de mi computador :)